### Preparar ambiente

In [1]:
import sys
!{sys.executable} -m pip install PyAthena

### Conectar con pyathena

In [2]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata' # Database in Glue
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

### Completar con columnas lat_frecuentes, long_frecuentes y lat_final, long_final

In [3]:
fecha_inicio = '09/01/2020'
fecha_final = '09/30/2020'
fechas = pd.date_range(fecha_inicio,fecha_final)

for fecha in fechas:
    day = str(fecha.day)
    if len(day) == 1:
        day = f'0{day}'
    codigo_fecha = str(fecha.month) + day
    print(codigo_fecha)
    
    tabla_usuarios_distancia = f'todos_usuarios_delta_dist_{codigo_fecha}'
    
    # Tabla historica preliminar para capturar latitud y longitud frecuente (mes base: 0 +1)
    tabla_historico_usuarios = 'historico_usuarios_preliminar'
    
    query_usuario_freq = f'''CREATE VIEW hogares_frecuentes AS
                                SELECT t_res.caid,
                                         t_res.iso_country_code,
                                         t_res.first_lat,
                                         t_res.first_long
                                FROM 
                                    (SELECT t_aux.caid AS caid,
                                         t_aux.iso_country_code AS iso_country_code,
                                         FIRST_VALUE(t_aux.lat_hogar) over( partition by t_aux.caid,
                                         t_aux.iso_country_code) AS first_lat,
                                         FIRST_VALUE(t_aux.long_hogar) over( partition by t_aux.caid,
                                         t_aux.iso_country_code) AS first_long
                                    FROM 
                                        (SELECT t_hist.caid,
                                         t_hist.iso_country_code,
                                         t_hist.lat_hogar,
                                         t_hist.long_hogar,
                                         COUNT(*) AS count_hogar
                                        FROM {tabla_usuarios_distancia} AS t_ud
                                        LEFT JOIN {tabla_historico_usuarios} AS t_hist
                                            ON t_ud.caid = t_hist.caid
                                                AND t_ud.iso_country_code = t_hist.iso_country_code
                                        GROUP BY  t_hist.caid, t_hist.iso_country_code, t_hist.lat_hogar, t_hist.long_hogar
                                        ORDER BY  COUNT(*) DESC, SUM(t_hist.n_obs_hogar) DESC) AS t_aux ) AS t_res
                                    GROUP BY  t_res.caid, t_res.iso_country_code, t_res.first_lat, t_res.first_long'''
                               

    cursor.execute(query_usuario_freq)
    time.sleep(6)
    while (not cursor.has_result_set):
        time.sleep(6)
    cursor.close()
    
    tabla_usuarios_distancia_cor = f'todos_usuarios_delta_dist_corregida_{codigo_fecha}'
  
    query = f'''CREATE TABLE {tabla_usuarios_distancia_cor} AS 
                SELECT t_users.*,
                    t_hogares.first_lat AS lat_frecuente,
                    t_hogares.first_long AS long_frecuente,  
                    CASE 
                    WHEN t_users.lat_hogar is NULL THEN
                    t_hogares.first_lat
                    ELSE t_users.lat_hogar END as lat_final,
                    CASE 
                    WHEN t_users.long_hogar is NULL THEN
                    t_hogares.first_long
                    ELSE t_users.long_hogar END as long_final
                FROM {tabla_usuarios_distancia} AS t_users
                LEFT JOIN hogares_frecuentes AS t_hogares
                    ON t_users.caid = t_hogares.caid
                    AND t_users.iso_country_code = t_hogares.iso_country_code'''

    cursor.execute(query)
    cursor.execute("DROP VIEW graphdata.hogares_frecuentes")

901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
